## Michael Ersevim - DSC680 - Project 1 Spam detector and remover for any email account

Starting to attach to email accounts with imapClient

This block builds the list of Real emails

In [ ]:
from imap_tools import MailBox, AND

user = input("email address? ")
password = input("password? ")

email_list = []

# Get subject and text of all emails from cleaned INBOX folder to compile a list of good (ham) emails

with MailBox('imap.cox.net').login(user, password) as mailbox:

#    for msg in mailbox.fetch(AND('SINCE 05-Apr-2023')): ## Works to limit new emails since date listed
#    for msg in mailbox.fetch(AND(charset='utf8','SINCE 05-Apr-2023')):
    for msg in mailbox.fetch(charset='utf8'):
        print(msg.subject, msg.text)
        email_list.append(msg.text) #attempting to append the text to a list for SVM training purposes
        

This block builds the list of Spam emails

In [ ]:
from imap_tools import MailBox, AND

user = input("email address? ")
password = input("password? ")

junk_email_list = []

# Get subject and text of all emails from cleaned JUNK folder to compile a list of bad (spam) emails

with MailBox('imap.cox.net').login(user, password, initial_folder='JUNK') as mailbox:

#    for msg in mailbox.fetch(AND('SINCE 05-Apr-2023')): ## Works to limit new emails since date listed
#    for msg in mailbox.fetch(AND(charset='utf8','SINCE 05-Apr-2023')):
    for msg in mailbox.fetch(charset='utf8'):
        print(msg.subject, msg.text)
        junk_email_list.append(msg.text) #attempting to append the text to a list for SVM training purposes
        

In [ ]:
#Make a folder(s) in the email account named PySpam for segregating Spam emails

user = input("email address? ")
password = input("password? ")

with MailBox('imap.cox.net').login(user, password) as mailbox:
    mailbox.folder.create('INBOX|PySpam') ### Use the first time to create a subfolder for future spam emails


In [ ]:
#Check to see the created folder(s) are in the email account you are working on

user = input("email address? ")
password = input("password? ")

with MailBox('imap.cox.net').login(user, password) as mailbox:
    for f in mailbox.folder.list('INBOX'): ###Use this loop to check for all subfolders
        print(f)  # FolderInfo(name='INBOX|cats', delim='|', flags=('\\Unmarked', '\\HasChildren'))


In [ ]:
email_list = [email_list]
print(email_list)

In [ ]:
type(email_list)

Training an SVM model to discern Spam from Ham

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn import metrics


In [ ]:
# Actual email junk/real for future appending
spam = pd.read_csv('myspam.csv', encoding='cp1252')

# Sample dataset for proof of concept/testing
#spam = pd.read_csv('spam.csv')

Note - the training csv ('myspam.csv') file is a very small sample size 
currently, given I have not been able to properly 
automate the importing of 'legible, parsible' emails

In [ ]:
spam.head()

In [ ]:
x = spam['v2']
y = spam["v1"]
x_train, x_test,y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [ ]:
cv = CountVectorizer()
features = cv.fit_transform(x_train)

In [ ]:
model = svm.SVC()
model.fit(features,y_train)

In [ ]:
features_test = cv.transform(x_test)
print("Accuracy: {}".format(model.score(features_test,y_test)))

Ad Hoc testing below of trained model

In [ ]:
new_x = "Jean Greene passed away yesterday. She and Joe have been First Church members for several years. " # ad hoc test of potential email texts...

In [ ]:
new_x = [new_x]

In [ ]:
new_features_test = cv.transform(new_x)

In [ ]:
new_pred = model.predict(new_features_test)

In [ ]:
print(new_pred)

Loop to check for prediction and move email based on it

In [ ]:
user = input("email address? ")
password = input("password? ")

# Use SVM to predict on email and move to PySpam folder if determined to be Spam
with MailBox('imap.cox.net').login(user, password) as mailbox:
    for msg in mailbox.fetch(charset='utf8'):
        new_email_test = cv.transform(msg)
        new_pred = model.predict(new_email_test)
        if new_pred = 'Spam':
            mailbox.move(msg, 'INBOX/PySpam')

        

* Word Cloud Maker - Helpful in seeing the word drivers of choosing Spam or Ham

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud


In [ ]:
#nltk.download('stopwords') #Only needs this the first time - computer retains the downloaded list

In [ ]:
df = pd.read_csv('myspam.csv', encoding='cp1252') # Special encoding prevents errors from certain quote symbols

In [ ]:
punctuations_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '', punctuations_list)
    return text.translate(temp)
 
df['v2']= df['v2'].apply(lambda x: remove_punctuations(x))
df.head()

In [ ]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')

    imp_words = []

    # Storing the important words
    for word in str(text).split():
        word = word.lower()

        if word not in stop_words:
            imp_words.append(word)

    output = " ".join(imp_words)

    return output


df['v2'] = df['v2'].apply(lambda text: remove_stopwords(text))
df.head()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

def plot_word_cloud(data, typ):
    email_corpus = " ".join(data['v2'])
 
    plt.figure(figsize=(10, 10))
    wc = WordCloud(background_color='white',
                   max_words=100,
                   width=200,
                   height=100,
                   collocations=False).generate(email_corpus)
 
    plt.title(f'WordCloud for {typ} emails.', fontsize=15)
    plt.axis('off')
    plt.imshow(wc)
    plt.show()
    print()
 
 
plot_word_cloud(df[df['v1'] == 'ham'], typ='Non - Spam')
plot_word_cloud(df[df['v1'] == 'spam'], typ='Spam')

In [ ]:
print(df['v2'])